In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob
import os

plt.style.use('ggplot') # Make the graphs a bit prettier

In [2]:
data = {}
files = glob.glob(os.path.expanduser("~/dev/ma/data/results/seq_louvain/*.json")) + glob.glob(os.path.expanduser("~/dev/ma/data/results/dlm_all/*.json")) + glob.glob(os.path.expanduser("~/dev/ma/data/results/plm_all/*.json"))

for path in files:
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

algorithm_names = {
    'thrill partitioned louvain': 'partitioned local moving',
    'thrill node based fully distributed local moving': 'synchronous local moving'
}

frames['algorithm_run'].replace({ 'algorithm': algorithm_names }, inplace=True)

frames['program_run']['graph_path'] = frames['program_run']['graph']

graph_names = {
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000-sorted-preprocessed-*.bin': 'LFR 100K', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_1000000-preprocessed-*.bin': 'LFR 1M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_10000000-preprocessed-*.bin': 'LFR 10M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_100000000-preprocessed-*.bin': 'LFR 100M', 
    '/home/kit/iti/kp0036/graphs/mu-04/graph_50_10000_mu_0.4_200000000-sorted-preprocessed-*.bin': 'LFR 200M', 
    '/home/kit/iti/kp0036/graphs/hypercubegraph23-preprocessed-*.bin': 'hypercube', 
    '/home/kit/iti/kp0036/graphs/uk-2002.metis-preprocessed-*.bin': 'uk-2002', 
    '/home/kit/iti/kp0036/graphs/uk-2007-05.metis-preprocessed-*.bin': 'uk-2007-05', 
    '/home/kit/iti/kp0036/graphs/in-2004.metis-preprocessed-*.bin': 'in-2004', 
    '/home/kit/iti/kp0036/graphs/com-friendster-preprocessed-*.bin': 'com-friendster', 
    '/home/kit/iti/kp0036/graphs/com-lj.ungraph-preprocessed-*.bin': 'com-lj', 
    '/home/kit/iti/kp0036/graphs/com-orkut.ungraph-preprocessed-*.bin': 'com-orkut', 
    '/home/kit/iti/kp0036/graphs/com-youtube.ungraph-preprocessed-*.bin': 'com-youtube', 
    '/home/kit/iti/kp0036/graphs/com-amazon.ungraph-preprocessed-*.bin': 'com-amazon'
}

frames['program_run'].replace({ 'graph': graph_names }, inplace=True)

def calc_worker_label(x):
    if np.isnan(x['hosts']):
        return 'sequential'
    else: 
        return str(int(x['hosts'])) + 'x' + str(int(x['workers_per_host']))
        
frames['program_run']['worker_label'] = frames['program_run'].apply(calc_worker_label, axis=1)

In [16]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['algorithm', 'worker_label', 'graph'])[['runtime', 'modularity']].mean() \
    .stack() \
    .unstack(['algorithm', 'worker_label'])

algorithm                 partitioned local moving                         \
worker_label                                 16x16        2x16       4x16   
graph                                                                       
LFR 100K       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 100M       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 10M        runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
LFR 1M         runtime                         NaN         NaN  49.133987   
               modularity                      NaN         NaN   0.592719   
LFR 200M       runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-amazon     runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-friendster runtime                 1442.971659         NaN        NaN   
               modularity                 0.616655         NaN        NaN   
com-lj         runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-orkut      runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
com-youtube    runtime                         NaN  150.616430        NaN   
               modularity                      NaN    0.713472        NaN   
hypercube      runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
in-2004        runtime                         NaN         NaN  68.560918   
               modularity                      NaN         NaN   0.980307   
uk-2002        runtime                         NaN         NaN        NaN   
               modularity                      NaN         NaN        NaN   
uk-2007-05     runtime                 1185.680737         NaN        NaN   
               modularity                 0.996166         NaN        NaN   

algorithm                             sequential louvain  \
worker_label                     8x16         sequential   
graph                                                      
LFR 100K       runtime            NaN           1.438680   
               modularity         NaN           0.521836   
LFR 100M       runtime            NaN                NaN   
               modularity         NaN                NaN   
LFR 10M        runtime     225.192853        1018.980000   
               modularity    0.599273           0.599285   
LFR 1M         runtime            NaN          28.929400   
               modularity         NaN           0.591128   
LFR 200M       runtime            NaN                NaN   
               modularity         NaN                NaN   
com-amazon     runtime            NaN           1.024390   
               modularity         NaN           0.925548   
com-friendster runtime            NaN        5733.810000   
               modularity         NaN           0.623746   
com-lj         runtime     222.109561          96.720000   
               modularity    0.744079           0.747613   
com-orkut      runtime     217.506169         170.680000   
               modularity    0.664707           0.665134   
com-youtube    runtime            NaN           6.388920   
               modularity         NaN           0.718879   
hypercube      runtime            NaN        1537.810000   
               modularity         NaN           0.572864   
in-2004        runtime            NaN          15.064000   
               modularity         NaN           0.98

In [8]:
frames['clustering'] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'], left_on='program_run_id', right_index=True) \
    .groupby(['algorithm', 'graph']).agg({ 'worker_label': 'first', 'runtime': 'mean', 'modularity': 'mean'}) \
    .stack() \
    .unstack(['algorithm']).unstack()

algorithm      partitioned local moving                          \
                                runtime modularity worker_label   
graph                                                             
LFR 100K                           None       None         None   
LFR 100M                           None       None         None   
LFR 10M                         225.193   0.599273         8x16   
LFR 1M                           49.134   0.592719         4x16   
LFR 200M                           None       None         None   
com-amazon                         None       None         None   
com-friendster                  1442.97   0.616655        16x16   
com-lj                           222.11   0.744079         8x16   
com-orkut                       217.506   0.664707         8x16   
com-youtube                     150.616   0.713472         2x16   
hypercube                          None       None         None   
in-2004                         68.5609   0.980307         4x16   
uk-2002                         324.351   0.989725         8x16   
uk-2007-05                      1185.68   0.996166        16x16   

algorithm      sequential louvain                          \
                          runtime modularity worker_label   
graph                                                       
LFR 100K                  1.43868   0.521836   sequential   
LFR 100M                     None       None         None   
LFR 10M                   1018.98   0.599285   sequential   
LFR 1M                    28.9294   0.591128   sequential   
LFR 200M                     None       None         None   
com-amazon                1.02439   0.925548   sequential   
com-friendster            5733.81   0.623746   sequential   
com-lj                      96.72   0.747613   sequential   
com-orkut                  170.68   0.665134   sequential   
com-youtube               6.38892   0.718879   sequential   
hypercube                 1537.81   0.572864   sequential   
in-2004                    15.064    0.98016   sequential   
uk-2002                   589.205    0.98977   sequential   
uk-2007-05                   None       None         None   

algorithm      synchronous local moving                          
                                runtime modularity worker_label  
graph                                                            
LFR 100K                        9.63083   0.524983         2x16  
LFR 100M                        640.933    0.56462        16x16  
LFR 10M                         151.686   0.596983         8x16  
LFR 1M                          27.1538   0.592534         4x16  
LFR 200M                        860.819   0.574363        32x16  
com-amazon                      35.5199   0.925443         2x16  
com-friendster                   1597.7   0.612749        16x16  
com-lj                          167.701   0.730967         8x16  
com-orkut                       132.774   0.657185         8x16  
com-youtube                     41.1524   0.720974         2x16  
hypercube                       115.434   0.539654         8x16  
in-2004                         98.1693    0.98012         4x16  
uk-2002                         221.951   0.989648         8x16  
uk-2007-05                      496.868    0.99619        16x16